In [27]:
import sys
import os
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import *
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *


import torch.nn.functional as F

import torch
import torch.nn as nn

    
class PrecisionLoss(torch.nn.Module):
    def __init__(self):
        super(PrecisionLoss, self).__init__()

    def forward(self, predictions, targets):
        # Compute precision
        precision = torch.sum(predictions * targets) / torch.sum(predictions)
        # Take the negative value as the loss to maximize precision
        loss = -precision
        return loss

class TorchModel(torch.nn.Module):
    def __init__(self, n_hidden_layers, units_in_first_layer, units_per_hidden_layer=100):
        super(TorchModel, self).__init__()
        units_per_layer = [units_in_first_layer]
        for layer_i in range(n_hidden_layers):
            units_per_layer.append(units_per_hidden_layer)
        units_per_layer.append(1)
        seq_args = []
        for layer_i in range(len(units_per_layer)-1):
            units_in = units_per_layer[layer_i]
            units_out = units_per_layer[layer_i+1]
            seq_args.append(
                torch.nn.Linear(units_in, units_out))
            if layer_i != len(units_per_layer)-2:
                seq_args.append(torch.nn.ReLU())
        self.stack = torch.nn.Sequential(*seq_args)

    def forward(self, feature_mat):
        return self.stack(feature_mat)


class NumpyData(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __getitem__(self, item):
        return self.features[item, :], self.labels[item]

    def __len__(self):
        return len(self.labels)


class MyCV:
    def __init__(self, estimator, param_grid, cv):
        self.estimator = estimator
        self.param_grid = param_grid
        self.cv = cv

    def fit(self, X, y):
        self.train_features = X
        self.train_labels = y
        self.best_params_ = {}
        np.random.seed(1)
        fold_vec = np.random.randint(
            low=0, high=self.cv, size=self.train_labels.size)
        
        best_mean_accuracy = 0
        loss_df_list = []
        for param_dict in self.param_grid:
            for param_name, [param_value] in param_dict.items():
                setattr(self.estimator, param_name, param_value)
            
            local_loss_df_list = []    
            for test_fold in range(self.cv):
                is_set_dict = {
                    "validation": fold_vec == test_fold,
                    "subtrain": fold_vec != test_fold,
                }
                set_features = {
                    set_name: self.train_features[is_set, :]
                    for set_name, is_set in is_set_dict.items()
                }
                set_labels = {
                    set_name: self.train_labels[is_set]
                    for set_name, is_set in is_set_dict.items()
                }
                self.estimator.fit(
                    X=set_features, y=set_labels)
                predicted_labels = self.estimator.predict(
                    X=set_features["validation"])
           
                local_loss_df_list.append(self.estimator.loss_df)
            mean_local_loss_df = pd.concat(local_loss_df_list).groupby(
                ["hidden_layers", "step_size", "epoch", "set_name"]).mean().reset_index()
            loss_df_list.append(mean_local_loss_df)         
        self.loss_mean_df = pd.concat(loss_df_list)
        print(self.loss_mean_df)

    def predict(self, X):
        return self.estimator.predict(X)


class RegularizedMLP:
    def __init__(self, max_epochs, batch_size, step_size, hidden_layers, units_per_hidden_layer):
        self.max_epochs = max_epochs
        self.batch_size = batch_size
        self.step_size = step_size
        self.hidden_layers = hidden_layers
        self.units_per_hidden_layer = units_per_hidden_layer
        # self.loss_fun = PrecisionLoss()
        self.loss_fun = torch.nn.BCEWithLogitsLoss()

    def fit(self, X, y):
        set_features = X
        set_labels = y
        # Preparing subtrain and validation data loaders
        subtrain_csv = NumpyData(
            set_features["subtrain"], set_labels["subtrain"])
        subtrain_dl = torch.utils.data.DataLoader(
            subtrain_csv, batch_size=self.batch_size, shuffle=True)
        loss_df_list = []
       
        model = TorchModel(self.hidden_layers, set_features["subtrain"].shape[1])
        model.train()
        optimizer = torch.optim.SGD(model.parameters(), lr=self.step_size)

        for epoch in range(self.max_epochs):
            for batch_features, batch_labels in subtrain_dl:
                # Take a step and compute prediction error
                # Compute prediction error
                pred_tensor = model(batch_features.float()).reshape(
                    len(batch_labels.float()))
                loss_tensor = self.loss_fun(
                    pred_tensor, batch_labels.float())
                # Backpropagation
                optimizer.zero_grad()
                loss_tensor.backward()
                optimizer.step()

            # then compute subtrain/validation loss.
            for set_name in set_features:
                feature_mat = set_features[set_name]
                label_vec = set_labels[set_name]
                feature_mat_tensor = torch.from_numpy(
                    feature_mat.astype(np.float32))
                label_vec_tensor = torch.from_numpy(
                    label_vec.astype(np.float32))

                pred_tensor = model(feature_mat_tensor.float()).reshape(
                    len(label_vec_tensor.float()))
                loss_tensor = self.loss_fun(
                    pred_tensor, label_vec_tensor.float())
                set_loss = loss_tensor.item()

                loss_df_list.append(pd.DataFrame({
                    "hidden_layers": self.hidden_layers,
                    "step_size": self.step_size,
                    "set_name": set_name,
                    "loss": set_loss,
                    "epoch": epoch,
                }, index=[0]))
        self.model = model
        self.loss_df = pd.concat(loss_df_list)

    def decision_function(self, X):
        self.model.eval()
        with torch.no_grad():
            return self.model(torch.Tensor(X)).numpy().ravel()

    def predict(self, X):
        return np.where(self.decision_function(X) > 0, 1, 0)



eurusd_df = pd.read_csv("/Users/newuser/Projects/robust-algo-trader/data/trades_seq_fixed_EURUSD_H1_2011_2023.csv")
eur_usd_labels = eurusd_df["label"]
eur_usd_features = eurusd_df[["position", "RSI", "ATR", "ADX", "AROON_Oscillator"]]


data_dict = {
    "eurusd": (eur_usd_features.to_numpy() , eur_usd_labels.to_numpy()),
}

param_list = [
    {
        'hidden_layers': [4],
        'step_size': [0.1],
    }
]

for data_set, (input_mat, output_vec) in data_dict.items():
    rmlp = RegularizedMLP(
        max_epochs=100,
        batch_size=100,
        step_size=0.1,
        hidden_layers=3,
        units_per_hidden_layer=100,
    )
    learner_instance = MyCV(estimator=rmlp, param_grid=param_list, cv=2)
    learner_instance.fit(input_mat, output_vec)
    pred_y = learner_instance.predict(input_mat)
    accuracy = accuracy_score(output_vec, pred_y)
    precision = precision_score(output_vec, pred_y)
    print(f"accuracy_score: {accuracy}")
    print(f"precision_score: {precision}")
    # loss_df = learner_instance.loss_mean_df
    # loss_df.index = range(len(loss_df))
    # print(loss_df)
    # loss_df.to_csv(out_file, encoding='utf-8', index=False)
print("done")



     hidden_layers  step_size  epoch    set_name      loss
0                4        0.1      0    subtrain  0.677485
1                4        0.1      0  validation  0.680048
2                4        0.1      1    subtrain  0.674962
3                4        0.1      1  validation  0.678351
4                4        0.1      2    subtrain  0.676438
..             ...        ...    ...         ...       ...
195              4        0.1     97  validation  0.680654
196              4        0.1     98    subtrain  0.671355
197              4        0.1     98  validation  0.678689
198              4        0.1     99    subtrain  0.669589
199              4        0.1     99  validation  0.680580

[200 rows x 5 columns]
accuracy_score: 0.582024643633728
precision_score: 0.47231833910034604
done


In [ ]:
spam_scaled_features

In [ ]:
eur_usd_features